In [0]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('/content/data.csv', delimiter=',', unpack = True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])
global_step = tf.Variable(0, trainable= False, name='global_step')

X= tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

### 텐서보드 사용하기

전과같이 데이터를 읽고 placeholder값을 설정해줍니다.


In [0]:
with tf.name_scope('layer1'):
  W1 = tf.Variable(tf.random_uniform([2,10],-1.,1.), name = 'W1')
  L1 = tf.nn.relu(tf.matmul(X,W1))

with tf.name_scope('layer2'):
  W2 = tf.Variable(tf.random_uniform([10,20],-1.,1.), name = 'W2')
  L2 = tf.nn.relu(tf.matmul(L1,W2)) 

with tf.name_scope('layer3'):
  W3 = tf.Variable(tf.random_uniform([20,3],-1.,1.), name = 'W3')
  model = tf.nn.relu(tf.matmul(L2,W3))

with tf.name_scope('optimizer'):
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = Y, logits = model))
  optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
  train_op = optimizer.minimize(cost, global_step = global_step)

```with tf.name_scope``` 를 이용해 텐서보드에서 한 계층 내부를 표현해줍니다. 그 후 ```name```으로 이름(W1)을 정해줍니다.   
**p.s** 이름은 변수,placeholder,각각의 연산,활성화 함수 등 모든 텐서에 붙일 수 있습니다.    


In [5]:
tf.summary.scalar('cost',cost)

<tf.Tensor 'cost:0' shape=() dtype=string>

```tf.summary```모듈의 ```scalar```함수를 이용해 값이 하나인 텐서를 수집합니다. 

In [0]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
  saver.restore(sess, ckpt.model_checkpoint_path)
else:
  sess.run(tf.global_variables_initializer())


In [0]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs',sess.graph)


```tf.summary.merge_all```함수로 앞서 지정한 텐서들을 수집한 다음   
``` tf.summary.FileWriter```함수를 이용해 그래프와 텐서들의 값을 저장할 디렉토리를 설정합니다.


In [14]:
for step in range(5):
  sess.run(train_op, feed_dict={X: x_data, Y: y_data})

  print('Step: %d'%sess.run(global_step), 'Cost: %.3f' %sess.run(cost, feed_dict={X:x_data, Y:y_data}))


Step: 11 Cost: 0.740
Step: 12 Cost: 0.713
Step: 13 Cost: 0.688
Step: 14 Cost: 0.666
Step: 15 Cost: 0.645


In [0]:
summary = sess.run(merged, feed_dict={X:x_data, Y:y_data})
writer.add_summary(summary, global_step=sess.run(global_step))

sess.run을 이용해 앞서 ```merged```로 모아둔 텐서의 값들을 계산하여 수집한뒤, ```writer.add_summary```함수를 이용해 해당 값들을 앞서 지정한 디렉터리에 저장합니다.   
또한 ```global_step```값을 이용해 수집한 시점을 기록해 둡니다.

In [16]:
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y,1)
print('예측값',sess.run(prediction, feed_dict={X:x_data}))
print('실제값',sess.run(target, feed_dict={Y:y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f'%sess.run(accuracy *100,feed_dict={X:x_data, Y:y_data}))



예측값 [0 1 1 2 0 0 2]
실제값 [0 1 1 2 0 0 2]
정확도: 100.00


모든 학습을 완료한 뒤 logs라는 디렉토리가 새로 생기면 맥이나 리눅스의 테미널 or 윈도우의 cmd에서 ```tensorboard --logdir = ./log```명령을 실행합니다.   
혹시 colab등에서 사용시 logs안의 파일을 다운받아 컴퓨터 내부 저장소로 이동해 ```logs```라는 폴더를 만들고 ```cd logs폴더가 있는 주소```를 한뒤 위 명령어를 실행해 줍니다.